In [1]:
!pip install trueskill
!pip install trueskillthroughtime

from collections import defaultdict
from community import community_louvain
from dataclasses import dataclass, field
import datetime
import itertools
import math
import networkx
import pandas
import random
import trueskill
import trueskillthroughtime as ttt
from typing import List, Optional, Tuple, Union
import urllib.request

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for trueskill: filename=trueskill-0.4.5-py3-none-any.whl size=18046 sha256=b6c56c07f9a01228bfaf244b237da05f04b8994fbff3981864936b8bc15d3fa1
  Stored in directory: /root/.cache/pip/wheels/bf/cb/b7/441dde3542ccda9474afc6b53b81da197b0867e56e0dae7e09
Successfully built trueskill
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for trueskillthroughtime: filename=trueskillthroughtime-1.0.0-py3-none-any.whl size=12866 sha256=da5a5b36ba8378b980a2ee21d3e1f61cb08fb98f434bc24da22005120f91e962
  Stored in directory: /root/.cache/pip/wheels/92/9c/a3/3957ff943d16583d78728cbc4fad728668447d7934ffd06a2a
Successfully built trueskillthroughtime


In [3]:
Player = str
SetResult = Tuple[int, int]
TwoSets = Tuple[SetResult, SetResult]
ThreeSets = Tuple[SetResult, SetResult, SetResult]
MatchResult = Union[TwoSets, ThreeSets]

@dataclass
class Match:
  date: datetime.date
  home: Tuple[Player, Player]
  away: Tuple[Player, Player]
  score: MatchResult

years = tuple([
    eval(
        urllib.request.urlopen(
            f'https://raw.githubusercontent.com/travis-scholtens/trueskill-by-division/main/{year}.py'
        ).read().decode('utf-8')
    ) for year in (2018, 2019)
])

In [4]:
home_won = lambda match: match.score[-1][0] > match.score[-1][1]

def predicted_by(match_delta):
  def predicted_outcome(match):
    delta = match_delta(match)
    if delta > 0:
      return 1
    if delta < 0:
      return -1
    return 0
  return predicted_outcome

def prediction_accuracy(matches, method, predicted_outcome):
  def calculate():
    correct = incorrect = total = 0
    for match in matches:
      predicted = predicted_outcome(match)
      actual = 1 if home_won(match) else -1
      if actual == predicted:
        correct += 1
      elif actual == -predicted:
        incorrect += 1
      total += 1
    return (correct/total, incorrect/total, (correct + incorrect)/total, correct/incorrect)
  (correct, incorrect, coverage, ratio) = calculate()
  print(f'{method}: {correct:.4f} correct, {incorrect:.4f} incorrect, {coverage:.2f} coverage, {ratio:.2f} win/loss')
  return correct

@dataclass
class HistoryInput:
  home_victory = (1,0)
  away_victory = (0,1)

  dates: list = field(default_factory=list)
  teams: list = field(default_factory=list)
  results: list = field(default_factory=list)

  def add(self, match):
    self.dates.append(match.date.toordinal())
    self.teams.append((match.home, match.away))
    self.results.append(self.home_victory if home_won(match) else self.away_victory)

In [5]:
def opponent_pairs(matches):
  edges = set()
  for match in matches:
    edges |= {tuple(sorted(ps)) for ps in itertools.product(match.home, match.away)}
  return edges

def partition(matches):
  return community_louvain.best_partition(networkx.Graph(opponent_pairs(matches)))

def filter_matches(matches, communities, predicate):
   return [match for match in matches if predicate(match, communities)]

new_players = lambda match, communities: (
    any([player not in communities for player in match.home + match.away])
)

no_new_players = lambda match, communities: not new_players(match, communities)

community_count = lambda match, communities: (
    len({communities[player] for player in match.home + match.away})
)

intra = lambda match, communities: (
    no_new_players(match, communities) and community_count(match, communities) == 1
)

inter = lambda match, communities: (
    no_new_players(match, communities) and community_count(match, communities) > 1
)

communities = partition(years[0])

subsets = [
    ('all matches', lambda _, __: True),
    ('with no new players', no_new_players),
    ('intradivision', intra),
    ('interdivision', inter),
    ('with new players', new_players),
]

summary = pandas.DataFrame(data={
    'Match population': [context for (context, _) in subsets],
    'N': [len(filter_matches(years[1], communities, predicate)) for (_, predicate) in subsets],
})

def accuracy_by_match_type(matches, communities, rating_type, delta):
  subset_success_rates = []
  for (context, predicate) in subsets:
    subset_success_rates.append(prediction_accuracy(filter_matches(matches, communities, predicate), f'{rating_type} ({context})', predicted_by(delta)))
  summary[rating_type] = subset_success_rates

print('\n'.join([
  (lambda count: f'{context}: {count} ({count/len(years[1]):.2f})')(len(filter_matches(years[1], communities, predicate)))
  for (context, predicate) in subsets
]))

all matches: 2218 (1.00)
with no new players: 1257 (0.57)
intradivision: 814 (0.37)
interdivision: 443 (0.20)
with new players: 961 (0.43)


In [6]:
def coin_flip_delta():
  random.seed(420)
  return lambda _: random.random() * 2 - 1

accuracy_by_match_type(years[1], communities, 'Coin flip', coin_flip_delta())

Coin flip (all matches): 0.4950 correct, 0.5050 incorrect, 1.00 coverage, 0.98 win/loss
Coin flip (with no new players): 0.5044 correct, 0.4956 incorrect, 1.00 coverage, 1.02 win/loss
Coin flip (intradivision): 0.4754 correct, 0.5246 incorrect, 1.00 coverage, 0.91 win/loss
Coin flip (interdivision): 0.4628 correct, 0.5372 incorrect, 1.00 coverage, 0.86 win/loss
Coin flip (with new players): 0.4839 correct, 0.5161 incorrect, 1.00 coverage, 0.94 win/loss


In [7]:
def elo(matches):
  elo = defaultdict(lambda: 1500)
  scale = 400
  k = 32

  team_average = lambda team: sum([elo[player] for player in team])
  expected = lambda rating_a, rating_b: 1/(1 + pow(10, (rating_b - rating_a)/scale))
  next_rating = lambda last_rating, expected, actual: last_rating + k * (actual - expected)
  for match in matches:
    delta = team_average(match.home) - team_average(match.away)
    expected = 1 if delta > 0 else -1 if delta < 0 else 0
    actual = 1 if home_won(match) else -1
    for player in match.home:
      elo[player] = next_rating(elo[player], expected, actual)
    for player in match.away:
      elo[player] = next_rating(elo[player], -expected, -actual)
  return {player: rating for (player, rating) in elo.items()}

def elo_delta(elo):
  team_average = lambda team: sum([elo[player] if player in elo else 1500 for player in team])
  return lambda match: team_average(match.home) - team_average(match.away)

accuracy_by_match_type(years[1], communities, 'Elo', elo_delta(elo(years[0])))

Elo (all matches): 0.5532 correct, 0.3584 incorrect, 0.91 coverage, 1.54 win/loss
Elo (with no new players): 0.5593 correct, 0.3691 incorrect, 0.93 coverage, 1.52 win/loss
Elo (intradivision): 0.5590 correct, 0.3649 incorrect, 0.92 coverage, 1.53 win/loss
Elo (interdivision): 0.5598 correct, 0.3770 incorrect, 0.94 coverage, 1.49 win/loss
Elo (with new players): 0.5453 correct, 0.3444 incorrect, 0.89 coverage, 1.58 win/loss


In [8]:
def win_percentages(matches):
  wins_losses = defaultdict(lambda: [0,0])
  for match in matches:
    (winners, losers) = (match.home, match.away) if home_won(match) else (match.away, match.home)
    for player in winners:
      wins_losses[player][0] += 1
    for player in losers:
      wins_losses[player][1] += 1
  return {player: wins/(wins + losses) for (player, (wins, losses)) in wins_losses.items()}

def win_percentage_delta(percents):
  team_average = lambda team: sum([percents[player] if player in percents else 0.5 for player in team])
  return lambda match: team_average(match.home) - team_average(match.away)

accuracy_by_match_type(years[1], communities, 'Win %',
                       win_percentage_delta(win_percentages(years[0])))

Win % (all matches): 0.5870 correct, 0.3999 incorrect, 0.99 coverage, 1.47 win/loss
Win % (with no new players): 0.6006 correct, 0.3970 incorrect, 1.00 coverage, 1.51 win/loss
Win % (intradivision): 0.6081 correct, 0.3894 incorrect, 1.00 coverage, 1.56 win/loss
Win % (interdivision): 0.5869 correct, 0.4108 incorrect, 1.00 coverage, 1.43 win/loss
Win % (with new players): 0.5692 correct, 0.4037 incorrect, 0.97 coverage, 1.41 win/loss


In [9]:
MU = 0
SIGMA = 4.6

def true_skill(matches):
  env = trueskill.TrueSkill(mu=MU, sigma=SIGMA, beta=0.43, tau=0.14, draw_probability=0)
  skill = defaultdict(env.create_rating)
  team_skills = lambda team: [skill[player] for player in team]
  for match in matches:
    teams = (match.home, match.away) if home_won(match) else (match.away, match.home)
    new_ratings = env.rate([team_skills(team) for team in teams])
    for (player, rating) in zip(teams[0] + teams[1], new_ratings[0] + new_ratings[1]):
      skill[player] = rating
  return {player: rating for (player, rating) in skill.items()}

# Generic, will work for trueskill.Gaussian or ttt.Gaussian skills
def distribution_delta(team_skill, match):
  (home, away) = [team_skill(team) for team in (match.home, match.away)]
  distribution = home.__class__(mu=home.mu - away.mu,
                                sigma=math.sqrt(home.sigma**2 + away.sigma**2))
  return distribution.mu/distribution.sigma
  
def true_skill_delta(skills):
  team_skill = lambda team: trueskill.Gaussian(mu=sum([skills[player].mu if player in skills else MU for player in team]),
                                               sigma=math.sqrt(sum([(skills[player].sigma if player in skills else SIGMA)**2 for player in team])))
  return lambda match: distribution_delta(team_skill, match)

accuracy_by_match_type(years[1], communities, 'TrueSkill', true_skill_delta(true_skill(years[0])))

TrueSkill (all matches): 0.6150 correct, 0.3819 incorrect, 1.00 coverage, 1.61 win/loss
TrueSkill (with no new players): 0.6158 correct, 0.3842 incorrect, 1.00 coverage, 1.60 win/loss
TrueSkill (intradivision): 0.6216 correct, 0.3784 incorrect, 1.00 coverage, 1.64 win/loss
TrueSkill (interdivision): 0.6050 correct, 0.3950 incorrect, 1.00 coverage, 1.53 win/loss
TrueSkill (with new players): 0.6139 correct, 0.3788 incorrect, 0.99 coverage, 1.62 win/loss


In [10]:
def converge(input):
  history = ttt.History(input.teams, input.results, input.dates, mu=MU, sigma=SIGMA, beta=0.43, gamma=0.014, p_draw=0)
  history.convergence(epsilon=0.01, iterations=25)
  return {player: curve[-1][1] for (player, curve) in history.learning_curves().items()}

def true_skill_through_time(matches):
  input = HistoryInput()
  for match in matches:
    input.add(match)
  return converge(input)

def true_skill_through_time_delta(skills):
  team_skill = lambda team: sum([skills[player] if player in skills else ttt.Gaussian(mu=MU, sigma=SIGMA) for player in team], ttt.N00)
  return lambda match: distribution_delta(team_skill, match)

accuracy_by_match_type(years[1], communities, 'TS through time', true_skill_through_time_delta(true_skill_through_time(years[0])))

Iteration =  0 , step =  (10.849392769919262, 3.5692857634228323)
Iteration =  1 , step =  (3.26785705320606, 0.8641493301768199)
Iteration =  2 , step =  (1.4084840561105314, 0.4246594277623785)
Iteration =  3 , step =  (0.7204591295762279, 0.1680717223469439)
Iteration =  4 , step =  (0.3722799402602337, 0.07745475121715462)
Iteration =  5 , step =  (0.20893990162472065, 0.047465744608095495)
Iteration =  6 , step =  (0.13847370808068638, 0.02729196544709911)
Iteration =  7 , step =  (0.100969471891724, 0.017609991627471233)
Iteration =  8 , step =  (0.07689646340249823, 0.012098357981848107)
Iteration =  9 , step =  (0.06131814934513535, 0.009639475470850645)
Iteration =  10 , step =  (0.05187435237429083, 0.008389617059252785)
Iteration =  11 , step =  (0.04447477697874325, 0.007357868862220229)
Iteration =  12 , step =  (0.03871706022723398, 0.006428134505915839)
Iteration =  13 , step =  (0.03444726018883415, 0.005605116709056723)
Iteration =  14 , step =  (0.03042495339175577, 0

In [11]:
def community_skill(matches, communities):
  inputs = defaultdict(HistoryInput)
  for match in matches:
    for community in {communities[player] for player in match.home + match.away}:
      inputs[community].add(match)
  return {community: converge(input) for (community, input) in inputs.items()}

def community_skill_delta(communities, community_skills):
  team_skill = lambda community, team: sum([
      community_skills[community][player]
      if player in community_skills[community] else ttt.Gaussian(mu=MU, sigma=SIGMA)
      for player in team], ttt.N00)

  def delta(match):
    match_community_counts = defaultdict(lambda: 0)
    for player in match.home + match.away:
      if player in communities:
        match_community_counts[communities[player]] += 1
    if not match_community_counts:
      return 0

    best_community = max(match_community_counts, key=match_community_counts.get)
    return distribution_delta(lambda team: team_skill(best_community, team), match)

  return delta

community_skills = community_skill(years[0], communities)
accuracy_by_match_type(years[1], communities, 'Division TTT',
                       community_skill_delta(communities, community_skills))

Iteration =  0 , step =  (7.931065952438894, 3.4640557535208174)
Iteration =  1 , step =  (2.1325658708075164, 0.696441882960565)
Iteration =  2 , step =  (0.9967628825729886, 0.25711734911136164)
Iteration =  3 , step =  (0.3276960836602152, 0.07669905068650129)
Iteration =  4 , step =  (0.1705369845098274, 0.03237069759231348)
Iteration =  5 , step =  (0.08392163982337131, 0.013360584808889797)
Iteration =  6 , step =  (0.05931240718359465, 0.00773350572116982)
Iteration =  7 , step =  (0.040631760539157624, 0.006066303449021593)
Iteration =  8 , step =  (0.02829595267187668, 0.004111473170326008)
Iteration =  9 , step =  (0.01995111133024996, 0.002829797030828196)
Iteration =  10 , step =  (0.014762331988272592, 0.001898471960320336)
Iteration =  11 , step =  (0.011320936617289834, 0.0015010129189856336)
Iteration =  12 , step =  (0.008983443902709998, 0.0014396829073057837)
End
Iteration =  0 , step =  (8.876023638732262, 3.550631568230337)
Iteration =  1 , step =  (3.3798090702643

In [12]:
def mean_skill(skills):
  return ttt.Gaussian(mu=sum([s.mu for s in skills])/len(skills),
                      sigma=math.sqrt(sum([s.sigma**2 for s in skills]))/len(skills))

def mean_skill_difference(from_skills, to_skills):
  if from_skills == to_skills:
    return ttt.N00
  overlap = set(from_skills) & set(to_skills)
  if not overlap:
    return None
  from_mean = mean_skill([from_skills[p] for p in overlap])
  to_mean = mean_skill([to_skills[p] for p in overlap])
  return from_mean - to_mean

def inter_community_conversions(community_values, community_skills):
  return {
    (from_community, to_community): mean_skill_difference(community_skills[from_community], community_skills[to_community])
    for (from_community, to_community)
    in itertools.product(community_values, community_values)
  }

def inter_community_skill_delta(communities, community_skills, conversions):
  team_skill = lambda community, team: sum(
      [community_skills[communities[player]][player] - conversions[(communities[player], community)]
       if player in communities else ttt.Gaussian(mu=MU, sigma=SIGMA)
       for player in team], ttt.N00)
  def delta(match):
    match_community_counts = defaultdict(lambda: 0)
    for player in match.home + match.away:
      if player in communities:
        match_community_counts[communities[player]] += 1
    if not match_community_counts:
      return 0

    best_community = max(match_community_counts, key=match_community_counts.get)
    return distribution_delta(lambda team: team_skill(best_community, team), match)
  return delta

accuracy_by_match_type(years[1], communities, 'Division TTT + conversion',
                       inter_community_skill_delta(communities, community_skills, inter_community_conversions(set(communities.values()), community_skills)))

Division TTT + conversion (all matches): 0.6312 correct, 0.3656 incorrect, 1.00 coverage, 1.73 win/loss
Division TTT + conversion (with no new players): 0.6420 correct, 0.3580 incorrect, 1.00 coverage, 1.79 win/loss
Division TTT + conversion (intradivision): 0.6290 correct, 0.3710 incorrect, 1.00 coverage, 1.70 win/loss
Division TTT + conversion (interdivision): 0.6659 correct, 0.3341 incorrect, 1.00 coverage, 1.99 win/loss
Division TTT + conversion (with new players): 0.6171 correct, 0.3757 incorrect, 0.99 coverage, 1.64 win/loss


In [13]:
(summary.style
    .hide_index()
    .highlight_max(axis=1,
                   subset=summary.columns[2:-2],
                   props={'font-style': 'italic'}.items())
    .highlight_max(axis=1,
                   subset=summary.columns[2:],
                   props={'font-weight': 'bold'}.items())
    .format(precision=4)
)

Match population,N,Coin flip,Elo,Win %,TrueSkill,TS through time,Division TTT,Division TTT + conversion
all matches,2218,0.4950,0.5532,0.5870,0.6150,0.6145,0.6195,0.6312
with no new players,1257,0.5044,0.5593,0.6006,0.6158,0.6261,0.6269,0.6420
intradivision,814,0.4754,0.5590,0.6081,0.6216,0.6241,0.6290,0.6290
interdivision,443,0.4628,0.5598,0.5869,0.6050,0.6298,0.6230,0.6659
with new players,961,0.4839,0.5453,0.5692,0.6139,0.5994,0.6098,0.6171
